In [260]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from collections import Counter
from sklearn.metrics import classification_report
import numpy as np
from sklearn.tree import export_text

In [261]:
ds = pd.read_pickle("../../data/interim/final_merged_data.pkl");

#uncomment to remove free games, can muddy up the results sometimes
ds = ds[ds['f2p_flag'] == False].copy()
ds = ds.drop(columns=['f2p_flag'])


#drop title not a attribute we are looking for
ds = ds.drop(['title'], axis=1)
ds = ds.drop(['app_id'], axis=1)

ds['release'] = pd.to_datetime(ds['release'])
ds['release_month'] = ds['release'].dt.month
ds = ds.drop(['release'], axis=1)

ds



,peak_players,win,mac,linux,user_reviews,Categories,Genres,Tags,Price,release_year,copies_sold_reviews_proxy,estimated_revenue,log_estimated_revenue,avg_sentiment,has_sentiment,release_month
0,4529,True,False,False,35131,"Single-player,Multi-player,Co-op,Shared/Split ...","Action,Indie","Indie,Casual,Sports,Bowling",19.99,2023,1053930,21068060.7,16.863269,NaN,0,1
1,168191,True,False,False,55455,"Single-player,Steam Achievements,Full controll...","Action,Adventure","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",39.99,2023,1663650,66529363.5,18.013154,NaN,0,3
4,6132,True,False,False,16025,"Single-player,Steam Achievements,Full controll...",Action,"Indie,Adventure,Nudity,Violent,Sexual Content,...",29.99,2023,480750,14417692.5,16.483967,NaN,0,1
6,8564,True,False,False,12160,"Single-player,Multi-player,Co-op,Online Co-op,...","Action,Adventure,Casual,Indie,Early Access","Tower Defense,Rogue-lite,RTS,Replay Value,Perm...",9.99,2023,364800,3644352.0,15.108689,NaN,0,4
7,1227,True,False,False,1964,"Single-player,Steam Achievements,Steam Cloud,F...","Action,Casual,Indie,Simulation,Strategy",NaN,5.99,2023,58920,352930.8,12.774030,NaN,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37486,137,True,False,False,360,"Single-player,Multi-player,Family Sharing",Strategy,"Stealth,Puzzle-Platformer,Immersive Sim,Action...",2.99,2006,21600,64584.0,11.075737,0.242236,1,10
37487,15,True,False,False,250,"Single-player,Multi-player,Family Sharing",Strategy,"Indie,Casual,Strategy,Action,Match 3,Mouse only",1.19,2006,15000,17850.0,9.789815,NaN,0,7
37488,22,True,False,False,98,"Single-player,Family Sharing","Casual,Indie","Strategy,Action,Adventure,Survival",4.99,2006,5880,29341.2,10.286782,0.269231,1,12
37489,13,True,False,False,126,"Single-player,Multi-player,Family Sharing",Strategy,"Strategy,Turn-Based Strategy",4.99,2006,7560,37724.4,10.538089,0.169231,1,10


In [262]:
#Split categories into binary values, ex 1 or 0  for Action. DO this for every categories. Yes I know its alot of columns
ds['Categories'] = ds['Categories'].fillna('')
ds['Categories'] = ds['Categories'].str.split(',')
ds['Categories'] = ds['Categories'].apply(lambda x: [c.strip() for c in x if c.strip() != ''])
mlb_cat = MultiLabelBinarizer()

categories_encoded = pd.DataFrame(
    mlb_cat.fit_transform(ds['Categories']),
    columns=[f"category_{cat}" for cat in mlb_cat.classes_],
    index=ds.index
)
ds = pd.concat([ds, categories_encoded], axis=1)
ds = ds.drop(['Categories'], axis=1)


In [263]:
#Split Genres into binary values
ds['Genres'] = ds['Genres'].fillna('')
ds['Genres'] = ds['Genres'].str.split(',')
ds['Genres'] = ds['Genres'].apply(lambda x: [g.strip() for g in x if g.strip() != ''])
mlb_genres = MultiLabelBinarizer()

genres_encoded = pd.DataFrame(
    mlb_genres.fit_transform(ds['Genres']),
    columns=[f"genre_{g}" for g in mlb_genres.classes_],
    index=ds.index
)
ds = pd.concat([ds, genres_encoded], axis=1)
ds = ds.drop(['Genres'], axis=1)

In [264]:
#Split tags into binary values, only the 50 most commons because there are 100's of tags. We already have enough columns
ds['Tags'] = ds['Tags'].fillna('')
ds['Tags'] = ds['Tags'].str.split(',')
ds['Tags'] = ds['Tags'].apply(
    lambda x: [t.strip() for t in x if t.strip() != '']
)
tag_counter = Counter(tag for tags_list in ds['Tags'] for tag in tags_list)
top_50_tags = [tag for tag, count in tag_counter.most_common(50)]
for tag in top_50_tags:
    ds[f"tag_{tag}"] = ds['Tags'].apply(lambda tags_list: 1 if tag in tags_list else 0)
ds = ds.drop(['Tags'], axis=1)



In [265]:
# We create a binary value for the tree to work
#in this case its profitablity, 1 = proftiable 0 = not proftable 
# 
# A decision tree classifier doesn’t predict a continuous number like revenue — it predicts categories.
#To discover what factors cause profitability, you need a clear definition of what “profitable” means.

#Target var based on average revenue, top 50%
ds['profit_flag'] = (ds['estimated_revenue'] > ds['estimated_revenue'].median()).astype(int)

#If you want it base on a custom revenue
#ds['profit_flag'] = (ds['estimated_revenue'] > 1000000).astype(int)





In [ ]:
#what overall attributes predict 
colsToDrop = [
    'profit_flag',
    'estimated_revenue',
    'log_estimated_revenue',
    'copies_sold_reviews_proxy',
    'peak_players',
    'user_reviews'
]

game_features = ds.drop( colsToDrop, axis=1)


#Target Outcome
profit_labels = ds['profit_flag']

testDataPercentage = 0.2

#training data     , the test data     , data profitable  , test profitable = 
game_features_train, game_features_test, profit_labels_train, profit_labels_test = \
    train_test_split(game_features, profit_labels, test_size= testDataPercentage, random_state=13)

clf = DecisionTreeClassifier(max_depth=5, random_state=14)
clf.fit(game_features_train, profit_labels_train)

rules = export_text(clf, feature_names=list(game_features_train.columns), max_depth=5)
print(rules)


pred = clf.predict(game_features_test)
print(classification_report(profit_labels_test, pred))

pd.DataFrame({
    'feature': game_features_train.columns,
    'importance': clf.feature_importances_
}).sort_values(by='importance', ascending=False)




|--- Price <= 7.07
|   |--- avg_sentiment <= 0.14
|   |   |--- category_Steam Trading Cards <= 0.50
|   |   |   |--- Price <= 3.24
|   |   |   |   |--- Price <= 1.17
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- Price >  1.17
|   |   |   |   |   |--- class: 0
|   |   |   |--- Price >  3.24
|   |   |   |   |--- release_year <= 2014.50
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- release_year >  2014.50
|   |   |   |   |   |--- class: 0
|   |   |--- category_Steam Trading Cards >  0.50
|   |   |   |--- release_year <= 2017.50
|   |   |   |   |--- Price <= 3.12
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- Price >  3.12
|   |   |   |   |   |--- class: 0
|   |   |   |--- release_year >  2017.50
|   |   |   |   |--- Price <= 1.44
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- Price >  1.44
|   |   |   |   |   |--- class: 1
|   |--- avg_sentiment >  0.14
|   |   |--- Price <= 3.02
|   |   |   |--- release_year <= 2013.50
|   |   |   |   |--- avg_sentiment 

,feature,importance
3,Price,0.680137
5,avg_sentiment,0.188877
39,category_Steam Trading Cards,0.084630
4,release_year,0.035936
36,category_Steam Cloud,0.005916
...,...,...
118,tag_Point & Click,0.000000
119,tag_Stylized,0.000000
120,tag_Free to Play,0.000000
121,tag_Gore,0.000000


In [267]:
#Classification basesd on ONLY Tags, what tags predict profitabilty
#----------------------------------

colsToDrop = [
    'profit_flag',
    'estimated_revenue',
    'log_estimated_revenue',
    'copies_sold_reviews_proxy',
    'user_reviews',
    'peak_players'
]

game_features = ds.drop( colsToDrop, axis=1)


tag_cols = [col for col in game_features.columns if col.startswith("tag_")]

# Remove free to play, hard to indicate revenue with :/
tag_cols = [col for col in tag_cols if col != "tag_Free to Play"]

X = game_features[tag_cols];  
y = profit_labels

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=13
)


clf = DecisionTreeClassifier(max_depth=12, random_state=13)
clf.fit(X_train, y_train)


pred = clf.predict(X_test)
print(classification_report(y_test, pred))

importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': clf.feature_importances_
})

# Filter to only tag columns
tag_importance = importance_df[importance_df['feature'].str.startswith("tag_")]

tag_importance.sort_values(by='importance', ascending=False)





              precision    recall  f1-score   support

           0       0.50      0.45      0.47      3051
           1       0.50      0.54      0.52      3057

    accuracy                           0.50      6108
   macro avg       0.50      0.50      0.50      6108
weighted avg       0.50      0.50      0.50      6108



,feature,importance
11,tag_Story Rich,0.031011
4,tag_Adventure,0.030953
23,tag_Shooter,0.029663
1,tag_Singleplayer,0.028565
3,tag_Casual,0.028430
7,tag_Simulation,0.027662
29,tag_Difficult,0.027500
5,tag_2D,0.027488
9,tag_Puzzle,0.027111
8,tag_RPG,0.026344


In [268]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

plt.figure(figsize=(160, 100))
plot_tree(clf, feature_names=game_features.columns, filled=True, max_depth=2)
plt.show()
